# 02 – TinyConvNet
Implement `TinyConvNet` and experiment with il training loop.

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

test_dataset = datasets.MNIST(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

batch_size = 64

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False
)

print("Train samples:", len(train_dataset))
print("Test samples:", len(test_dataset))

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()

        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))  # [B,6,14,14]
        x = self.pool(self.relu(self.conv2(x)))  # [B,16,5,5]
        x = x.view(x.size(0), -1)                # [B,400]
        x = self.relu(self.fc1(x))               # [B,120]
        x = self.relu(self.fc2(x))               # [B,84]
        x = self.fc3(x)                          # [B,10]
        return x

# TODO: Implement TinyConvNet
#   - Conv2d(1 -> 4, kernel_size=3, padding=1) + ReLU + MaxPool2d(2)
#   - Conv2d(4 -> 8, kernel_size=3, padding=1) + ReLU + MaxPool2d(2)
#   - Global Average Pooling (AdaptiveAvgPool2d((1,1)))
#   - Linear(8 -> 10)

class TinyConvNet(nn.Module):
    def __init__(self):
        super(TinyConvNet, self).__init__()
        # TODO: definire gli strati
        # self.conv1 = ...
        # self.conv2 = ...
        # self.pool = ...
        # self.relu = ...
        # self.gap = ...
        # self.fc = ...
        raise NotImplementedError("TODO: implement TinyConvNet constructor")

    def forward(self, x):
        # TODO: definire la forward
        # 1) Conv1 -> ReLU -> pool
        # 2) Conv2 -> ReLU -> pool
        # 3) Global Average Pooling
        # 4) Flatten
        # 5) Fully connected finale
        raise NotImplementedError("TODO: Implement TinyConvNet forward")

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# TODO: Chose the model to train
MODEL_NAME = "lenet"

if MODEL_NAME == "lenet":
    model = LeNet().to(device)
elif MODEL_NAME == "tiny":
    model = TinyConvNet().to(device)
else:
    raise ValueError("MODEL_NAME deve essere 'lenet' o 'tiny'.")

print(f"\n>>> Modello selezionato: {MODEL_NAME}")
print(model)
print("Numero parametri:", count_parameters(model))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 3  # TODO: Try to change epochs number

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)

    print(
        f"Epoch [{epoch+1}/{num_epochs}] "
        f"- Train loss: {train_loss:.4f}, acc: {train_acc*100:.2f}% "
        f"- Test loss: {test_loss:.4f}, acc: {test_acc*100:.2f}%"
    )
